In [1]:
#Importo las librerías necesarias
import pandas as pd

In [3]:
#Cargo el dataset de lesiones de 1951-2023
print(f"Cargando archivo original...")
try:
    df_hasta_2023 = pd.read_csv("C:\\Users\\FranciscoJH\\Downloads\\NBA Player Injury Stats(1951 - 2023).csv")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo 'C:\\Users\\FranciscoJH\\Downloads\\NBA Player Injury Stats(1951 - 2023).csv'. Asegúrate de que esté en la misma carpeta.")
    exit()

print(f"Datos cargados. Se encontraron {len(df_hasta_2023)} filas.")

Cargando archivo original...
Datos cargados. Se encontraron 37667 filas.


In [4]:
# Conversión de Fecha
# Nos aseguramos de que la columna 'Date' sea de tipo fecha
columna_fecha = 'Date'
# 'errors=coerce' convertirá cualquier fecha rota en 'NaT' (Not a Time)
print(f"Convirtiendo la columna '{columna_fecha}' a formato de fecha...")
df_hasta_2023[columna_fecha] = pd.to_datetime(df_hasta_2023[columna_fecha], errors='coerce')

Convirtiendo la columna 'Date' a formato de fecha...


In [5]:
# Eliminamos filas donde la fecha no se pudo leer
df_hasta_2023.dropna(subset=[columna_fecha], inplace=True)

In [6]:
# Extraemos el año de la fecha y filtramos
print("Filtrando datos entre 2010 y 2023...")
df_filtrado_hasta_2023 = df_hasta_2023[(df_hasta_2023[columna_fecha].dt.year >= 2010) & (df_hasta_2023[columna_fecha].dt.year <= 2023)]
#Además elimino la primer columna que almacena el ID del registro, ya que el dataset de lesiones de 2016-2025 no la tiene
df_filtrado_hasta_2023 = df_filtrado_hasta_2023.iloc[:, 1:]

Filtrando datos entre 2010 y 2023...


In [7]:
#Guardo el dataset filtrado
print(f"Filtrado completo. Se encontraron {len(df_filtrado_hasta_2023)} filas.")
df_filtrado_hasta_2023.to_csv("C:\\Users\\FranciscoJH\\Downloads\\NBA_Player_Injury_Stats_Filtrado_2023.csv", index=False)

Filtrado completo. Se encontraron 22998 filas.


In [8]:
#Ahora cargo el dataset de lesiones de 2016-2025
print(f"Cargando archivo original...")
try:
    df_hasta_2025 = pd.read_csv("C:\\Users\\FranciscoJH\\Downloads\\injury_data.csv")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo 'C:\\Users\\FranciscoJH\\Downloads\\injury_data.csv'. Asegúrate de que esté en la misma carpeta.")
    exit()

print(f"Datos cargados. Se encontraron {len(df_hasta_2025)} filas.")

Cargando archivo original...
Datos cargados. Se encontraron 16873 filas.


In [9]:
# Conversión de Fecha
# Nos aseguramos de que la columna 'Date' sea de tipo fecha
columna_fecha = 'Date'
# 'errors=coerce' convertirá cualquier fecha rota en 'NaT' (Not a Time)
print(f"Convirtiendo la columna '{columna_fecha}' a formato de fecha...")
df_hasta_2025[columna_fecha] = pd.to_datetime(df_hasta_2025[columna_fecha], errors='coerce')

Convirtiendo la columna 'Date' a formato de fecha...


In [10]:
# Eliminamos filas donde la fecha no se pudo leer
df_hasta_2025.dropna(subset=[columna_fecha], inplace=True)

In [11]:
# Extraemos el año de la fecha y filtramos
print("Filtrando datos entre 2023 y 2025...")
df_filtrado_hasta_2025 = df_hasta_2025[(df_hasta_2025[columna_fecha].dt.year >= 2023) & (df_hasta_2025[columna_fecha].dt.year <= 2025)]

Filtrando datos entre 2023 y 2025...


In [12]:
#Guardo el dataset filtrado
print(f"Filtrado completo. Se encontraron {len(df_filtrado_hasta_2025)} filas.")
df_filtrado_hasta_2025.to_csv("C:\\Users\\FranciscoJH\\Downloads\\NBA_Player_Injury_Stats_Filtrado_2025.csv", index=False)

Filtrado completo. Se encontraron 3955 filas.


In [13]:
#Ahora juntos los dos datasets
df_total = pd.concat([df_filtrado_hasta_2023, df_filtrado_hasta_2025], ignore_index=True)

#Elijo las columnas clave para eliminar los duplicados de año 2023
columnas_clave = ['Date', 'Team', 'Acquired', 'Relinquished', 'Notes']

#Elimino duplicados basándome en las columnas clave
#'keep=' controla cuál duplicado se queda:
#'last':  Se queda con el último registro de los duplicados que ve.
lesiones_csv = df_total.drop_duplicates(subset=columnas_clave, keep='last')

#Guardo el dataset final
lesiones_csv.to_csv("C:\\Users\\FranciscoJH\\Downloads\\NBA_Player_Injury_Stats_2010_2025.csv", index=False)
print(f"¡Éxito! Archivo final guardado en 'C:\\Users\\FranciscoJH\\Downloads\\NBA_Player_Injury_Stats_2010_2025.csv'")

¡Éxito! Archivo final guardado en 'C:\Users\FranciscoJH\Downloads\NBA_Player_Injury_Stats_2010_2025.csv'


In [14]:
#El dataset no tiene el ID del jugador lesionado, algo que es necesario para relacionarlo con otros datasets.
#Por lo tanto ahora voy a agregar el ID del jugador utilizando el dataset de jugadores de la NBA
# --- 0. Cargo el dataset de jugadores ---
df_player = pd.read_csv('C:\\Users\\FranciscoJH\\Downloads\\player.csv')


# --- 1. Unificar Nombres en df_lesiones ---
# Creamos una columna temporal 'player_name'
# Usamos .fillna() para "rellenar" los nulos de 'Acquired' con los valores de 'Relinquished'
# Esto nos da una sola columna con el nombre del jugador, sin importar la acción.
lesiones_csv['player_name_temp'] = lesiones_csv['Acquired'].fillna(lesiones_csv['Relinquished'])

# Quitamos los '•', por si hay alguno
lesiones_csv['player_name_temp'] = lesiones_csv['player_name_temp'].str.strip('• ')

print(f"Columna 'player_name_temp' creada en df_lesiones.")


# --- 2. Preparar df_player ---
# Seleccionamos solo las columnas que necesitamos para la fusión
# (Esto evita duplicar columnas como 'first_name', 'last_name', etc.)
df_player_mapa = df_player[['id', 'full_name']]


# --- 3. Fusionar (Merge) ---
# Unimos df_lesiones con df_player_mapa usando nuestras columnas de nombres
# how='left' asegura que mantengamos TODAS las lesiones, 
# incluso si un jugador no se encuentra en df_player (quedará como NaN)
df_lesiones_con_id = pd.merge(
    lesiones_csv,
    df_player_mapa,
    left_on='player_name_temp', # La columna unificada de df_lesiones
    right_on='full_name',     # La columna de nombre de df_player
    how='left'
)

print("Merge completado.")

# --- 4. Limpiar el DataFrame Final ---
# Renombramos la columna 'id' (que vino de df_player) a 'player_id'
df_lesiones_con_id.rename(columns={'id': 'player_id'}, inplace=True)

# Eliminamos las columnas temporales/duplicadas que ya no necesitamos
df_lesiones_con_id = df_lesiones_con_id.drop(columns=['player_name_temp', 'full_name'])

#Verifico el porcentaje de nulos en player_id
total_filas = len(df_lesiones_con_id)
nulos_player_id = df_lesiones_con_id['player_id'].isnull().sum()
porcentaje_nulos = (nulos_player_id / total_filas) * 100

print(f"Porcentaje de nulos en 'player_id': {porcentaje_nulos:.2f}%")

Columna 'player_name_temp' creada en df_lesiones.
Merge completado.
Porcentaje de nulos en 'player_id': 10.71%


C:\Users\FranciscoJH\AppData\Local\Temp\ipykernel_7900\4113195975.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lesiones_csv['player_name_temp'] = lesiones_csv['Acquired'].fillna(lesiones_csv['Relinquished'])
C:\Users\FranciscoJH\AppData\Local\Temp\ipykernel_7900\4113195975.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lesiones_csv['player_name_temp'] = lesiones_csv['player_name_temp'].str.strip('• ')


In [15]:
#Como vemos, hay un porcentaje de nulos del 10.71% en la columna player_id.
#Esto puede deberse a que algunos nombres en el dataset de lesiones no coinciden exactamente con los del dataset de jugadores,
#por ejemplo, diferencias en apodos, errores tipográficos, o formatos distintos (como incluir iniciales, sufijos, etc.).
#Para mejorar la coincidencia, podríamos implementar técnicas de limpieza de datos más avanzadas.
#Pero primero, veremos que nombres no tienen un ID asignado.

print("--- Investigando Nombres que Fallaron el Merge (Versión Corregida) ---")

# 1. Filtramos las filas donde player_id es nulo
df_fallos = df_lesiones_con_id[df_lesiones_con_id['player_id'].isnull()]

# --- 2. LA CORRECCIÓN ---
# Como 'player_name_temp' ya no existe, recreamos la lógica
# usando las columnas originales ('Acquired' y 'Relinquished')
# que SÍ están en 'df_fallos'.

# Rellenamos los nulos de 'Acquired' con los de 'Relinquished'
nombres_unificados_fallidos = df_fallos['Acquired'].fillna(df_fallos['Relinquished'])

# Limpiamos (el '•' y los espacios, por si hay algunos)
nombres_unificados_fallidos = nombres_unificados_fallidos.str.strip('• ')


# 3. Obtenemos la lista de nombres ÚNICOS que fallaron
nombres_fallidos = nombres_unificados_fallidos.value_counts()

print(f"Se encontraron {len(nombres_fallidos)} nombres únicos que no tuvieron coincidencia.")
print("\n--- Top 20 Nombres Fallidos (y cuántas veces aparecen) ---")
print(nombres_fallidos.head(20))

--- Investigando Nombres que Fallaron el Merge (Versión Corregida) ---
Se encontraron 232 nombres únicos que no tuvieron coincidencia.

--- Top 20 Nombres Fallidos (y cuántas veces aparecen) ---
Acquired
(William) Tony Parker                            80
Mike Conley Jr.                                  67
Emanuel Ginobili / Manu Ginobili                 66
James Michael McAdoo / James McAdoo (Michael)    65
Wesley Matthews / Wes Matthews Jr.               63
Marcus Morris                                    59
Maximilian Kleber / Maxi Kleber                  55
John Wall (Hildred)                              54
Ogugua Anunoby / O.G. Anunoby                    43
Nene / Nene Hilario / Maybyner Hilario           43
Metta World Peace / Ron Artest                   42
Raulzinho Neto / Raul Neto                       40
Louis Williams / Lou Williams                    39
Maurice Harkless / Moe Harkless                  38
C.J. Miles                                       38
Byron Mullens / 

In [23]:
#Ahora voy a utilizar otra librería para hacer un matching más avanzado entre los nombres.
#La librería se llama 'fuzzywuzzy' y permite comparar cadenas de texto con técnicas de coincidencia difusa.
from fuzzywuzzy import process, fuzz 

# --- 1. Unificar Nombres en df_lesiones ---
df_lesiones_con_id['player_name_temp'] = df_lesiones_con_id['Acquired'].fillna(df_lesiones_con_id['Relinquished'])
df_lesiones_con_id['player_name_temp'] = df_lesiones_con_id['player_name_temp'].str.strip('• ')
print("Columna 'player_name_temp' creada.")

# --- 2. Preparar el "Mapa de Búsqueda" de df_player ---
# Convertimos a lista para que fuzzywuzzy trabaje más rápido
nombres_player = df_player['full_name'].tolist() 
mapa_ids_player = dict(zip(df_player['full_name'], df_player['id']))
print("Mapa de jugadores (para búsqueda) creado.")

# --- 3. La Función de Búsqueda ---
def encontrar_mejor_coincidencia(nombre_lesion, lista_nombres_player, mapa_ids):
    if pd.isna(nombre_lesion):
        return None
    try:
        # Usamos 'fuzz.token_set_ratio' que es bueno para nombres (ignora "Jr.", "II", etc.)
        mejor_coincidencia, score = process.extractOne(
            nombre_lesion, 
            lista_nombres_player, 
            scorer=fuzz.token_set_ratio, # Usamos un 'scorer' robusto
            score_cutoff=85 # Umbral de confianza del 85%
        )
        
        if score >= 85:
            return mapa_ids[mejor_coincidencia]
        else:
            return None # No estaba lo suficientemente seguro
    except:
        return None

# --- 4. Crear el Mapa de Traducción usando .unique() ---
print("Obteniendo nombres únicos de 'lesiones'...")
# Obtenemos la lista corta de nombres únicos a buscar
nombres_unicos_lesiones = df_lesiones_con_id['player_name_temp'].dropna().unique()

print(f"Se encontraron {len(nombres_unicos_lesiones)} nombres únicos para buscar (vs {len(df_lesiones_con_id)} filas totales).")
print("Creando mapa de traducción... (Esto puede tardar unos minutos, pero solo una vez)")

# Creamos el diccionario de traducción
traduccion_mapa = {}
for nombre_lesion in nombres_unicos_lesiones:
    id_encontrado = encontrar_mejor_coincidencia(nombre_lesion, nombres_player, mapa_ids_player)
    if id_encontrado is not None:
        traduccion_mapa[nombre_lesion] = id_encontrado

print(f"Mapa de traducción creado. Se encontraron {len(traduccion_mapa)} coincidencias.")

# --- 5. Aplicar el Mapa ---
print("Aplicando mapa de traducción a todo el DataFrame...")
# .map() es órdenes de magnitud más rápido que .apply()
df_lesiones_con_id['player_id'] = df_lesiones_con_id['player_name_temp'].map(traduccion_mapa)

print("¡Merge difuso completado!")

# --- 6. Verificación ---
total_filas = len(df_lesiones_con_id)
nulos_player_id = df_lesiones_con_id['player_id'].isnull().sum()
porcentaje_nulos = (nulos_player_id / total_filas) * 100

print(f"Porcentaje de nulos en 'player_id' (después del fuzzy merge): {porcentaje_nulos:.2f}%")

# Eliminamos las columnas temporales/duplicadas que ya no necesitamos
df_lesiones_con_id = df_lesiones_con_id.drop(columns=['player_name_temp'])


Columna 'player_name_temp' creada.
Mapa de jugadores (para búsqueda) creado.
Obteniendo nombres únicos de 'lesiones'...
Se encontraron 1564 nombres únicos para buscar (vs 26019 filas totales).
Creando mapa de traducción... (Esto puede tardar unos minutos, pero solo una vez)
Mapa de traducción creado. Se encontraron 1466 coincidencias.
Aplicando mapa de traducción a todo el DataFrame...
¡Merge difuso completado!
Porcentaje de nulos en 'player_id' (después del fuzzy merge): 1.40%


In [ ]:
#Como vemos que el porcentaje de nulos es bajo, por eso decidí eliminar esas filas, ya que si no tienen ID no podré relacionarlas con otros datasets.

filas_antes = len(df_lesiones_con_id)
print(f"Filas antes de la limpieza final: {filas_antes}")

# .dropna() elimina las filas donde la columna especificada ('player_id') es Nula
df_limpio_final = df_lesiones_con_id.dropna(subset=['player_id'])

filas_despues = len(df_limpio_final)
print(f"Filas después de la limpieza final: {filas_despues}")
print(f"Se eliminaron {filas_antes - filas_despues} filas (el 1.40%).")

# --- GUARDAR ESTE ARCHIVO ---
df_limpio_final.to_csv("C:\\Users\\FranciscoJH\\Desktop\\lesiones_listas.csv", index=False)

Filas antes de la limpieza final: 26019
Filas después de la limpieza final: 25654
Se eliminaron 365 filas (el 1.40%).
